In [ ]:
Urne = {"rot": 6, "blau": 3, "grün": 4}
zurücklegen = True
n = 3

Ereignisse = [['rot', 'blau', 'blau'], ["grün","blau","blau"], ["blau","rot","blau"], ["blau","grün","blau"], ["blau","blau","rot"], ["blau","blau","grün"], ["blau","blau","blau"]]

In [ ]:
from fractions import Fraction
from igraph import Graph, EdgeSeq
import itertools

nr_vertices = int((1-len(Urne)**(n+1))/(1-len(Urne))) # nr of verticies calculated by geometrical series, assumes symmetrical tree
v_label = list(Urne.keys()) * int((nr_vertices-1) / len(Urne)) # create the labels
v_label.insert(0, "- root") # add the root node
G = Graph.Tree(nr_vertices, len(Urne)) # n stands for children number
lay = G.layout_reingold_tilford(mode='in', root=[0])

position = {k: lay[k] for k in range(nr_vertices)}
Y = [lay[k][1] for k in range(nr_vertices)]
M = max(Y)

es = EdgeSeq(G) # sequence of edges
E = [e.tuple for e in G.es] # list of edges

L = len(position)
Xn = [position[k][0] for k in range(L)]
Yn = [2*M-position[k][1] for k in range(L)]
Xe = []
Ye = []
for edge in E:
    Xe+=[position[edge[0]][0],position[edge[1]][0], None]
    Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]

labels = v_label

# set the paths accordingly
allPaths = []
for i in range(n+1):
    allPaths.extend(list(itertools.product(Urne.keys(), repeat=i)))
G.vs["path"] = allPaths

# calculate the probabilities 
lastProb = [] # saving the last probability per node for labeling
allProbs = [1]
for i in range(1, nr_vertices): # calculate the number of nodes using geometric series, skip root node
    numerator, denominator = 1, 1
    Urne_copy = Urne.copy()
    for j, item in enumerate(G.vs[i]["path"]): # the idea is to trace the path back from the root, using the path vector
        numerator = numerator * Urne_copy[item]
        denominator = denominator * sum(Urne_copy.values())
        if j+1 == len(G.vs[i]["path"]):
            lastProb.append(Fraction(Urne_copy[item], sum(Urne_copy.values())))
        if not zurücklegen:
            Urne_copy[item] = Urne_copy[item]-1
    allProbs.append(Fraction(numerator, denominator))

G.vs["lastprob"] = lastProb
G.vs["prob"] = allProbs

In [ ]:
def make_annotations(pos, text, font_size=10, font_color='rgb(250,250,250)'):
    L=len(pos)
    annotations = []
    for k in range(L):
        annotations.append(
            dict(
                text=labels[k][0], # set the first letter of description to node
                x=pos[k][0], y=2*M-position[k][1],
                xref='x1', yref='y1',
                font=dict(color=font_color, size=font_size),
                showarrow=False)
        )
    
    # annotations for path probability
    for i in range(2, len(Xe), 3): # start off with the third value in the array and bypassing None types
        annotations.append(dict(
            text=f"{G.vs[int((i-2)/3)]['lastprob']}", 
            x=(Xe[i-1]+Xe[i-2])/2, y=(Ye[i-1]+Ye[i-2])/2,
            xref='x1', yref='y1',
            font=dict(color="black", size=font_size),
            showarrow=False)
        )

    # annotation for probability of last elements
    for i in range(nr_vertices - len(Urne)**n, nr_vertices):
        annotations.append(dict(
            text=f"{G.vs[i]['prob']}", 
            x=Xn[i], y=Yn[i]-0.15,
            xref='x1', yref='y1',
            font=dict(color="black", size=font_size),
            showarrow=False)
        )
    
    return annotations

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()
fig.add_trace(go.Scatter(x=Xe,
                   y=Ye,
                   mode='lines',
                   line=dict(color='rgb(210,210,210)', width=2),
                   opacity=0.8
                   ))
fig.add_trace(go.Scatter(x=Xn,
                  y=Yn,
                  mode='markers',
                  marker=dict(symbol='circle-dot',
                                size=18,
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=list(zip(allPaths, np.array(allProbs).astype(str))),
                  hoverinfo='text',
                  opacity=0.8
                  ))

axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            )

fig.update_layout(annotations=make_annotations(position, v_label),
              xaxis=axis,
              yaxis=axis,
              showlegend=False
              )
fig.show()

In [ ]:
prob = []
for item in Ereignisse:
    prob.append(G.vs.find(path=tuple(item))["prob"])
print(f"Summe der Wahrscheinlichkeiten der Ereignisse: \n{prob}\n= {sum(prob)} = {float(sum(prob))*100}%")